# Graded-Challenge 3

*by* **Annisa Ditasila**
*batch 017*

<hr>

**Spesific goals**
> Who
-	Pelanggan
-	Retail
-	Pengirim
-	Marketer
-	Data analis
-	Engineer (frontend engineer, backend engineer, QA, devops)
-	Admin data

> What
-	Peningkatan pada jumlah transaksi, total penjualan san aktif user dengan satus complete.

> Where
-	Ketika transaksi terjadi, maka pesanan akan dikemas di Gudang dan dikirim sesuai dengan alamat user.
When
-	Disetiap bulannya antara kuarter 1 hingga kuarter 3 di tahun 2022.
Why 
-	Peningkatan pada aktif user juga akan membawa peningkatan pada jumlah transaksi dan jumlah penjualan

> When
-	Disetiap bulannya antara kuarter 1 hingga kuarter 3 di tahun 2022.

> Why 
-	Peningkatan pada aktif user juga akan membawa peningkatan pada jumlah transaksi dan jumlah penjualan


**Measurable goals**
-	How much / how many? Peningkatan seharusnya diatas 0% dari bulan sebelumnya
-	How will I know when it is accomplished? Setelah menghitung dari jumlah penjualan, jumlah transaksi dan jumlah user di setiap akhir bulannya.


**Achievable goals**
-	How can I accomplish this goals? Dengan cara memperbaiki kualitas kinerja didalam perusahaan, misalnya seperti hubungan admin dengan retail, kemudian kinerja pengiriman, dan kualitas aplikasi.
-	How realistic the goal? Jika dilihat dari data, perkembangan menunjukan penjualan kearah postif. Maka goal bisa dikatakan realistis atau sesuai keadaan.


**Relevant goals**
-	Apakah ini bermanfaat bagi perusahan? Ya.
-	Apakah waktunya tepat? Ya.
-	Apakah sesuai dengan kebutuhan perusahaan? Ya.
-	Apakah saya memiliki pengetahuan yang tepat untuk mencapai tujuan ini? Ya, karena saya seorang data analyst.
-	Apakah dapat diterapkan dalam lingkungan social-ekonomi saat ini? Tidak.


**Time-bound Goal**
-	When? Disetiap bulannya antara kuarter 1 hingga kuarter 3 2022
-	What can I do six month from now? – mengawasi jalannya operasi pengiriman dan menganalisis trend pasar di Q1-Q3 2022
-	What can I do today? – meriset data sesuai dengan kebutuhan penelitian pasar.


<hr>

**SMART**

Admin memperbaiki kinerjanya dengan retail, developer mengembangkan aplikasi, dan staff pengiriman memperbaiki kinerja mereka sehingga tujuan yang dicapai adalah peningkatan transaksi, penjualan dan aktif user yang berstatus complete di setiap bulannya diantara Q1 – Q3.

<hr>

**Problem Statement**
	
  Karena The Look E-Commerce adalah salah satu perusahaan terbesar di Planet mars, maka dari segi finansial juga membutuhkan masukan lebih banyak investasi untuk meningkatkan kuallitas perusahaan. Tentunya dengan mempertimbangkan *produk kategori apa saja yang berpengaruh dalam penjualan dalam Q1-Q3 2022?*


<hr>

# DATA IMPORT

In [ ]:
from google.cloud import bigquery
from google.colab import auth

In [ ]:
auth.authenticate_user()
print('Authenticated')

project_id = "hacktiv8-369916"
client = bigquery.Client(project=project_id)

Authenticated


<HR>

# DATA EXPLORING

In [ ]:
client.query('''SELECT *
                FROM `bigquery-public-data.thelook_ecommerce.order_items`
                where status = 'Complete'
                limit 3''') \
      .to_dataframe()

,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
0,22879,15632,12645,28951,61696,Complete,2019-05-13 23:30:37+00:00,2019-05-13 12:12:00+00:00,2019-05-16 19:23:00+00:00,NaT,3.0
1,7366,5017,4043,15784,19883,Complete,2021-12-24 04:13:36+00:00,2021-12-26 10:15:00+00:00,2021-12-29 04:36:00+00:00,NaT,3.5
2,13894,9459,7671,14248,37400,Complete,2022-10-22 01:13:25+00:00,2022-10-23 22:07:00+00:00,2022-10-27 04:38:00+00:00,NaT,3.5


<hr>

# **POIN PENJABARAN**

In [ ]:
# 1. Jumlah transaksi yang berstatus complete pada setiap bulan pada Q1-Q3 2022.
client.query('''SELECT year,
                       month,
                       NTILE(3) OVER (ORDER BY year, month) AS quarter,
                       number_of_sales,
                       CONCAT(ROUND(((number_of_sales 
                                      / LAG(number_of_sales, 1) OVER (ORDER BY year, month)) 
                                      - 1) 
                                      * 100
                                    , 2)
                              , '%') as month_to_month,
                FROM (SELECT EXTRACT(year from created_at) as year,
                             EXTRACT(month from created_at) as month,
                             COUNT(order_id) as number_of_sales,
                      FROM `bigquery-public-data.thelook_ecommerce.orders`
                      WHERE status='Complete' and created_at
                       # BETWEEN equals to >= Q1 2022 AND <= Q3 2022
                       BETWEEN "2022-01-01" AND "2022-09-30"
                      # EXTRACT is an agg function, so we need to agg it with GROUP BY
                      GROUP BY year, month)
                ORDER BY year ASC, month ASC''') \
      .to_dataframe()
     

,year,month,quarter,number_of_sales,month_to_month
0,2022,1,1,987,None
1,2022,2,1,973,-1.42%
2,2022,3,1,1050,7.91%
3,2022,4,2,1146,9.14%
4,2022,5,2,1193,4.1%
5,2022,6,2,1330,11.48%
6,2022,7,3,1497,12.56%
7,2022,8,3,1648,10.09%
8,2022,9,3,1655,0.42%


**Insight 1:** 
> 
*Jika dilihat dari transaksi yang berstatus complete pada quarter 1 hingga quarter 3 di tahun 2022, ada penurunan pada bulan Februari. Pencapaian terbesar berada di bulan September yaitu 1655 transaksi complete. Tetapi jika dilihat dari persentase kenaikan dari setiap bulan, maka persentase tertinggi berada di bulan Juli sebesar 12,56%*

In [ ]:
# 2. Total penjualan yang berstatus complete dalam USD tiap bulan selama Q1 - Q3 di tahun 2022 


client.query('''SELECT year,
                       month,
                       NTILE(3) OVER (ORDER BY year, month) as quarter,
                       FORMAT("%'.2f", total_sales) as total_sales,
                       CONCAT(ROUND(((total_sales 
                                      / LAG(total_sales, 1) OVER (ORDER BY year, month)) 
                                      - 1)
                                      * 100
                                    , 2)
                              ,'%') as month_to_month
                FROM (SELECT EXTRACT(year from created_at) as year,
                             EXTRACT(month from created_at) as month,
                             SUM(sale_price) as total_sales
                      FROM `bigquery-public-data.thelook_ecommerce.order_items`
                      WHERE status='Complete'
                       AND created_at BETWEEN "2022-01-01" AND "2022-09-30"
                      GROUP BY year, month)
                ORDER BY year ASC, month ASC''') \
      .to_dataframe()

,year,month,quarter,total_sales,month_to_month
0,2022,1,1,"83,669.56",None
1,2022,2,1,"85,583.85",2.29%
2,2022,3,1,"89,764.70",4.89%
3,2022,4,2,"98,331.38",9.54%
4,2022,5,2,"103,878.66",5.64%
5,2022,6,2,"117,782.61",13.38%
6,2022,7,3,"127,322.91",8.1%
7,2022,8,3,"139,380.34",9.47%
8,2022,9,3,"145,042.43",4.06%


**Insight 2:** 
> 
*Jika dilihat dari transaksi yang berstatus complete pada quarter 1 hingga quarter 3 di tahun 2022, disetiap bulannya mengalami kenaikan. Kenaikan tertinggi pada bulan Juni sebesar 13.38%. Jika dilihat total penjualan terbesar berada di bulan Semptember yaitu sebesar $145,042.43*

In [ ]:
# 3. User aktif yang melakukan transaksi complete tiap bulan selama Q1 - Q3 di tahun 2022 

client.query('''SELECT year,
                       month,
                       NTILE(3) OVER (ORDER BY year, month) AS quarter,
                       num_of_user,
                       CONCAT(ROUND((((num_of_user 
                                       / LAG(num_of_user, 1) OVER (ORDER BY year, month)) 
                                       - 1)
                                       * 100)
                                    , 2)
                              , '%') AS month_to_month
                FROM (SELECT EXTRACT(YEAR FROM created_at) AS year,
                             EXTRACT(MONTH FROM created_at) AS month,
                             COUNT(DISTINCT user_id) AS num_of_user
                      FROM `bigquery-public-data.thelook_ecommerce.orders`
                      WHERE status='Complete'
                       AND created_at BETWEEN "2022-01-01" AND "2022-09-30"
                      GROUP BY year, month)
                ORDER BY year ASC, month ASC''') \
      .to_dataframe()

,year,month,quarter,num_of_user,month_to_month
0,2022,1,1,980,None
1,2022,2,1,967,-1.33%
2,2022,3,1,1043,7.86%
3,2022,4,2,1140,9.3%
4,2022,5,2,1185,3.95%
5,2022,6,2,1319,11.31%
6,2022,7,3,1481,12.28%
7,2022,8,3,1632,10.2%
8,2022,9,3,1633,0.06%


**Insight 3:** 
> 
*Jika dilihat dari transaksi yang berstatus complete pada quarter 1 hingga quarter 3 di tahun 2022, penurunan user aktif ada di bulan Februari yaitu sebesar 13 user atau -1.33%. Dan Peningkatan tertinggi berada di bulan Juli sebesar 162 user atau 12.28%*

In [ ]:
# 4. Kategori product yang paling banyak dibeli yang berstatus complete di tiap bulan pada Q1-Q3.

client.query('''SELECT year, month, category, num_of_product
                FROM (SELECT *,
                             DENSE_RANK() OVER (PARTITION BY year, month ORDER BY num_of_product DESC) as rank
                      FROM (SELECT EXTRACT(YEAR FROM created_at) AS year,
                                   EXTRACT(MONTH FROM created_at) AS month,
                                   b.category,
                                   COUNT(b.category) AS num_of_product
                            FROM `bigquery-public-data.thelook_ecommerce.order_items` AS a
                            INNER JOIN `bigquery-public-data.thelook_ecommerce.products` AS b
                             ON a.product_id=b.id
                            WHERE status='Complete'
                             AND created_at BETWEEN "2022-01-01" AND "2022-09-30"
                            GROUP BY year, month, b.category)
                     )
                WHERE rank=1
                ORDER BY year, month''') \
      .to_dataframe()

,year,month,category,num_of_product
0,2022,1,Intimates,113
1,2022,2,Jeans,99
2,2022,3,Intimates,111
3,2022,4,Tops & Tees,129
4,2022,5,Tops & Tees,126
5,2022,6,Intimates,133
6,2022,7,Intimates,171
7,2022,8,Intimates,174
8,2022,9,Tops & Tees,173


**Insight 4:** 
> 
*Jika dilihat dari kategori produknya,  intimates adalah produk yang paling laris di bulan Januari, Maret, Juni, Juli, dan Agustus. Kemudian Tops&Tees menjadi produk terlaris kedua yang berada di puncak pada bulan April, Mei, Agustus. Dan yang terakhir adalah Jeans menjadi produk paling laris di bulan Februari.*

In [ ]:
# 5. Mencari kategori produk yang paling mendapatkan pendapatan di tiap bulannya selama Q1-Q3 di 2022.

client.query('''SELECT *,
                FROM (SELECT *,
                            DENSE_RANK() OVER (PARTITION BY year, month ORDER BY total_sales DESC) as rank
                      FROM (SELECT EXTRACT(YEAR FROM created_at) AS year,
                                    EXTRACT(MONTH FROM created_at) AS month,
                                   b.category,
                                    SUM(sale_price) AS total_sales
                      FROM `bigquery-public-data.thelook_ecommerce.order_items` AS a
                      INNER JOIN `bigquery-public-data.thelook_ecommerce.products` AS b
                       ON a.product_id=b.id
                      WHERE status='Complete'
                       AND created_at BETWEEN "2022-01-01" AND "2022-09-30"
                      GROUP BY year, month, b.category)
                      )
                WHERE rank=1
                ORDER BY year, month''') \
      .to_dataframe()

,year,month,category,total_sales,rank
0,2022,1,Outerwear & Coats,9397.679977,1
1,2022,2,Outerwear & Coats,10477.670033,1
2,2022,3,Outerwear & Coats,10741.599985,1
3,2022,4,Jeans,12509.950014,1
4,2022,5,Outerwear & Coats,13824.359980,1
5,2022,6,Outerwear & Coats,17291.199995,1
6,2022,7,Outerwear & Coats,15214.740009,1
7,2022,8,Outerwear & Coats,16523.519974,1
8,2022,9,Jeans,17585.640030,1


**Insight 5:** 
> 
*Pada Q1-Q3 tahun 2022, Outer & Coats menjadi produk paling banyak mendapatkan pendapatan, kecuali di bulan April dan September, Jeans menjadi produk paling banyak mendapatkan pendapatan*

In [ ]:
# 6. Membandingkan jumlah transaksi dan total penjualan dengan jumlah user yang melakukan transaksu

client.query('''SELECT CORR(num_of_order, num_of_user) AS corr_num_of_order, 
                       CORR(total_sales, num_of_user) AS corr_total_sales
                FROM (SELECT EXTRACT(YEAR FROM created_at) AS year,
                             EXTRACT(MONTH FROM created_at) AS month,
                             COUNT(DISTINCT order_id) AS num_of_order,
                             SUM(sale_price) AS total_sales,
                             COUNT(DISTINCT user_id) AS num_of_user
                      FROM `bigquery-public-data.thelook_ecommerce.order_items`
                      GROUP BY year, month
                      ORDER BY year, month)''') \
      .to_dataframe()

,corr_num_of_order,corr_total_sales
0,0.995906,0.994478


**Insight 6:** 
> 
*Dari hasil korelasi diatas bisa disimpulkan bahwa tingginya angka korelasi menjelaskan semakin kuat keterikatan hubungan antara jumlah transaksi dan jumlah user yang melakukan transaksi.*

<hr>

# **Poin Analisis**
Dari hasil analisis, disimpulkan bahwa:
-	Pada jumlah transaksi yang terjadi di Q1-Q3 peningkatan terbesar pada bulan Juni dan Juli sebesar 11.48% dan 12.56% jika dihitung dari jumlah transaksi bulan sebelumnya. Jumlah transaksi minus di bulan Februari. Dan jumlah transaksi bernilai positif di bulan Maret, Mei dan September walaupun tidak signifikan.
-	Pada total penjualan, peningkatan terbesar di bulan Juni sebesar 13.38%. Disetiap bulan mengalami kenaikan kearah positif
-	Peningkatan persentase aktif user meningkat signifikan pada bulan Juni dan Juli yaitu sebesar 11,31% dan 12,28% dari bulan sebelumnya, tetapi jika dilihat secara keseluruhan dari setiap bulan jumlah user aktif terus meningkat.
-	Di bulan Februari, walaupun aktif user berkurang, jumlah transaksi berkurang, tetapi jumlah penjualan tidak menurun.
-	Pada bulan januari dari setiap aktif user, jumlah transaksi dan jumlah penjualan terjadi penurunan, ini berarti hubungan antara mereka ada linier.


<hr>

# **Poin Pertanyaan**
1.	Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!

> Ya, Jika dilihat dari ranking produk nomor 1, mereka menempati 10% penjualan dari total penjualan.


2.	Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250000, apakah masuk akal?

> Jika dilihat per-kuarter dari kuarter sebelumnya, maka penargetan sebesar $250000 bisa dicapai.

3.	CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!

> Jumlah transaksi per kuarter:
1.	Q1 2022: 3010 orders
2.	Q2 2022: 3669 orders / 21.89% 
3.	Q3 2022: 4800 orders / 30.83% 

> Jumlah aktif user:
1.	Q1 2022: 2945 active users
2.	Q2 2022: 3598 active users / 22.17% 
3.	Q3 2022: 4649 active users / 29.21% 

> Perkembangan GMV per kuarter ($):
1.	Q1 2022: 259,018.11
2.	Q2 2022: 319,992.65 / 23.54%
3.	Q3 2022: 411,745.68 / 28.57% 

> Perusahaan layak mendapatkan investor baru, dikarenakan transaksi, aktif user
dan GMV menunjukan peningkatan yang signifikan disetiap kuarter pada tahun 2022. 

In [ ]:
# Jumlah transaksi disetiap kuarter
client.query('''SELECT year,
                       quarter,
                       number_of_sales,
                       CONCAT(ROUND(((number_of_sales 
                                      / LAG(number_of_sales, 1) OVER (ORDER BY year, quarter)) 
                                      - 1) 
                                      * 100
                                    , 2)
                              , '%') AS month_to_month,
                FROM (SELECT EXTRACT(YEAR FROM created_at) AS year,
                             EXTRACT(QUARTER FROM created_at) AS quarter,
                             COUNT(DISTINCT order_id) AS number_of_sales,
                      FROM `bigquery-public-data.thelook_ecommerce.orders`
                      WHERE status='Complete' AND created_at
                       # BETWEEN equals to >= Q1 2022 AND <= Q3 2022
                       BETWEEN "2022-01-01" AND "2022-09-30"
                      # EXTRACT is an agg function, so we need to agg it with GROUP BY
                      GROUP BY year, quarter)
                ORDER BY year ASC, quarter ASC''') \
      .to_dataframe()

,year,quarter,number_of_sales,month_to_month
0,2022,1,3010,None
1,2022,2,3669,21.89%
2,2022,3,4800,30.83%


In [ ]:
# Jumlah user aktif di setiap kuarter
client.query('''SELECT year,
                       quarter,
                       num_of_user,
                       CONCAT(ROUND(((num_of_user 
                                      / LAG(num_of_user, 1) OVER (ORDER BY year, quarter)) 
                                      - 1) 
                                      * 100
                                    , 2)
                              , '%') AS month_to_month,
                FROM (SELECT EXTRACT(YEAR FROM created_at) AS year,
                             EXTRACT(QUARTER FROM created_at) AS quarter,
                             COUNT(DISTINCT user_id) AS num_of_user,
                      FROM `bigquery-public-data.thelook_ecommerce.orders`
                      WHERE status='Complete' AND created_at
                       # BETWEEN equals to >= Q1 2022 AND <= Q3 2022
                       BETWEEN "2022-01-01" AND "2022-09-30"
                      # EXTRACT is an agg function, so we need to agg it with GROUP BY
                      GROUP BY year, quarter)
                ORDER BY year ASC, quarter ASC''') \
      .to_dataframe()

,year,quarter,num_of_user,month_to_month
0,2022,1,2945,None
1,2022,2,3598,22.17%
2,2022,3,4649,29.21%


In [ ]:
# Total penjualan dan GMV disetiap quarter
client.query('''SELECT year,
                       quarter,
                       FORMAT("%'.2f", total_sales) as total_sales,
                       CONCAT(ROUND(((total_sales 
                                      / LAG(total_sales, 1) OVER (ORDER BY year, quarter)) 
                                      - 1) 
                                      * 100
                                    , 2)
                              , '%') AS month_to_month,
                FROM (SELECT EXTRACT(YEAR FROM created_at) AS year,
                             EXTRACT(QUARTER FROM created_at) AS quarter,
                             SUM(sale_price) AS total_sales,
                      FROM `bigquery-public-data.thelook_ecommerce.order_items`
                      WHERE status='Complete' AND created_at
                       # BETWEEN equals to >= Q1 2022 AND <= Q3 2022
                       BETWEEN "2022-01-01" AND "2022-09-30"
                      # EXTRACT is an agg function, so we need to agg it with GROUP BY
                      GROUP BY year, quarter)
                ORDER BY year ASC, quarter ASC''') \
      .to_dataframe()

,year,quarter,total_sales,month_to_month
0,2022,1,"259,018.11",None
1,2022,2,"319,992.65",23.54%
2,2022,3,"411,745.68",28.67%
